# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
#from google_drive_downloader import GoogleDriveDownloader as gdd
#gdd.download_file_from_google_drive(file_id='1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL', dest_path='./data', unzip=True)

Unzipping...Done.


In [0]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [0]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In [22]:
#print (os.listdir('./'))
# print (os.listdir('../input/final-data/neural_nets/Project_data'))
# print (os.listdir('../input/keras-pretrained-models'))

['.config', 'Project_data', 'data', 'model_init_2018-12-3005_40_47.426425', 'sample_data']


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [0]:
train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 16

In [24]:
train_doc[0]

'WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1\n'

In [25]:
from collections import Counter
length = []
sizes = []
for folder in train_doc:
    imgs = os.listdir('/notebooks/storage/Final_data/Collated_training/train/'+ folder.split(';')[0])
    length.append(len(imgs))
    image =imread('/notebooks/storage/Final_data/Collated_training/train/'+ folder.split(';')[0]+'/'+imgs[0])
    sizes.append(image.shape)
len_cntr = Counter(length)
siz_cntr = Counter(sizes)
print (len_cntr)
print (siz_cntr)

Counter({30: 663})
Counter({(120, 160, 3): 479, (360, 360, 3): 184})


In [0]:
img_size = (120,160,3)

In [27]:
sizes = []
for folder in train_doc:
    imgs = os.listdir('/notebooks/storage/Final_data/Collated_training/train/'+ folder.split(';')[0])
    image =imread('/notebooks/storage/Final_data/Collated_training/train/'+ folder.split(';')[0]+'/'+imgs[0])
    image = imresize(image,(120,160,3))
    sizes.append(image.shape)
siz_cntr = Counter(sizes)
siz_cntr

Counter({(120, 160, 3): 663})

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [0]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [1,5,8,10,12,14,16,18,20,22,25,29]#create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y = img_size[0]
    z = img_size[1]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = imresize(image,(y,z,3))
                    
                    batch_data[folder,idx,:,:,0] =(image[:,:,0] - np.mean(image[:,:,0]))/np.std(image[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1]))/np.std(image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2]))/np.std(image[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        batch = batch+1
        rem_data = batch_size*batch
        for folder in range(len(t)-rem_data): # iterate over the batch_size 
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes
                image = imresize(image,(y,z,3))

                batch_data[folder,idx,:,:,0] =(image[:,:,0] - np.mean(image[:,:,0]))/np.std(image[:,:,0]) #normalise and feed in the image
                batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1]))/np.std(image[:,:,1])#normalise and feed in the image
                batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2]))/np.std(image[:,:,2])#normalise and feed in the image

            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        print (batch_data.shape,batch_labels.shape)
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [29]:
curr_dt_time = datetime.datetime.now()
train_path = '/notebooks/storage/Final_data/Collated_training/train'#'/notebooks/storage/Final_data/Collated_training/train'
val_path = '/notebooks/storage/Final_data/Collated_training/val' #'/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 5 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 5


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [30]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import * 
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet_v2 import MobileNetV2
model1 = TimeDistributed(MobileNetV2(weights = 'imagenet',include_top = False,pooling='avg'))

9412608/9406464 [==============================] - 2s 0us/step


In [31]:
from keras import layers
inp = layers.Input((12,120,160,3))
x = model1(inp)
x = layers.Dropout(0.5)(x)
x = layers.GRU(256,activation='relu',kernel_regularizer = l2(0.01),recurrent_regularizer=l2(0.01))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(5,activation='softmax',kernel_regularizer = l2(0.01))(x)
model = Model(input=inp,output=x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 12, 120, 160, 3)   0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 12, 1280)          2257984   
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 1280)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               1180416   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 3,439,685
Trainable params: 3,405,573
Non-trainable params: 34,112
_____________________________________________________________

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [32]:
optimiser = optimizers.SGD(lr=0.01)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 12, 120, 160, 3)   0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 12, 1280)          2257984   
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 1280)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               1180416   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 3,439,685
Trainable params: 3,405,573
Non-trainable params: 34,112
_____________________________________________________________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [0]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [0]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.4,patience=2, min_lr=0.001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [0]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [36]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/5
Source path =  Source path = ./Project_data/train/  ./Project_data/val/ ; batch size = 16
; batch size = 16
(16, 12, 120, 160, 3) (16, 5)
42/42 [==============================] - 66s 2s/step - loss: 13.7685 - categorical_accuracy: 0.3170 - val_loss: 13.2095 - val_categorical_accuracy: 0.3839

Epoch 00001: saving model to model_init_2018-12-3006_14_06.540593/model-00001-13.76847-0.31696-13.20948-0.38393.h5
(16, 12, 120, 160, 3) (16, 5)
Epoch 2/5
42/42 [==============================] - 43s 1s/step - loss: 12.8486 - categorical_accuracy: 0.6354 - val_loss: 12.5173 - val_categorical_accuracy: 0.7143

Epoch 00002: saving model to model_init_2018-12-3006_14_06.540593/model-00002-12.84862-0.63542-12.51731-0.71429.h5
Epoch 3/5
42/42 [==============================] - 50s 1s/step - loss: 12.2844 - categorical_accuracy: 0.7961 - val_loss: 12.2270 - val_categorical_accuracy: 0.6429

Epoch 00003: saving model to model_init_2018-12-3006_14_06.540593/model-00003-12.28438-0.79613-12.22704-